Phase 2 - Notebook 3 (copied from Databricks, not all code may work correctly in other IDEs). 

This notebook converts the updated case data from the previous script into FHIR format, and then sends each patient bundle separately through the API to MDH. 

In [ ]:
%pip install fhir.resources

In [ ]:
%run "/Shared/InfectiousDiseases/Syphilis/FHIR/Production/fhir_functions_p2"

In [ ]:
#https://pypi.org/project/fhir.resources/
#Default FHIR version is 5. Use version 4(R4B) by importing a subpackage.
from fhir.resources.R4B.patient import Patient
from fhir.resources.R4B.careplan import CarePlan
from fhir.resources.R4B.observation import Observation
from fhir.resources.R4B.questionnaireresponse import QuestionnaireResponse
from fhir.resources.R4B.bundle import Bundle
from fhir.resources.R4B.bundle import BundleEntry

import pandas as pd
import json
import os

In [ ]:
bt = "/dbfs/mnt/phmdw/Refined/PublicHealth/InfectiousDiseases/SyphilisProvisional/FHIR Base Tables/"

Read in updated cases.

In [ ]:
#For now just read these files in. Will eventually add some logic to grab the correct one.
updated_cases = pd.read_parquet(bt + "fhir_main_2024-07-12.parquet")
updated_cases = updated_cases[updated_cases['update_any'] == 1]

treatment_history = pd.read_parquet(bt + "fhir_treat_2024-07-12.parquet").dropna()

Read in relevant reference tables.

In [ ]:
reference = '/dbfs/mnt/phmdw/Trusted/PublicHealth/Reference/FHIR/'

language = pd.read_csv(reference + 'fhir_language_codes.csv')
orientation = pd.read_csv(reference + 'fhir_sexual_orientation.csv')

For each patient, create bundle and send.

In [ ]:
patients = updated_cases.case_id.unique()
print(str(len(patients)) + ' patient bundles to send.')

for patid in patients:
  
    #Create bundle structure
    patient_bundle = Bundle.construct()
    patient_bundle.type = 'searchset'
    bundle_list = []

    #Filter patient data
    temp = updated_cases[updated_cases['case_id'] == patid]
    temp_trt = treatment_history[treatment_history['case_id'] == patid]
    temp_trt = temp_trt.reset_index()

    #Create patient resource
    pat = create_patient_resource(temp)
    
    #Add patient resource to bundle
    pat_rg = Patient(**pat)
    pat_entry = BundleEntry.construct()
    pat_entry.fullUrl = 'urn:us-mn-state-health-medss-patient:' + temp.case_id.item()
    pat_entry.resource = pat_rg
    bundle_list.append(pat_entry)

    #Create observation resource
    obs = create_observation_resource(temp)
    
    #Add observation resource to bundle
    obs_rg = Observation(**obs)
    obs_entry = BundleEntry.construct()
    obs_entry.fullUrl = 'urn:us-mn-state-health-medss-observation:' + temp.case_id.item()
    obs_entry.resource = obs_rg
    bundle_list.append(obs_entry)

    #Create CarePlan resource(s) (if data is available, may need to change)
    if len(temp_trt) != 0:
        for x in range(0, len(temp_trt)):
            single_trt = temp_trt.iloc[x]

            cp = create_careplan_resource(single_trt)
    
            #Add CarePlan resource to bundle
            cp_rg = CarePlan(**cp)
            cp_entry = BundleEntry.construct()
            cp_entry.fullUrl = 'urn:us-mn-state-health-medss-careplan:' + str(x+1) + single_trt.case_id #add a number to this. must be unqiue if there are multiple resources shared
            cp_entry.resource = cp_rg
            bundle_list.append(cp_entry)

    #Create QuestionnaireResponse resource
    qr = create_questionnaire_resource(temp)

    #Add QuestionnaireResponse resource to bundle
    qr_rg = QuestionnaireResponse(**qr)
    qr_entry = BundleEntry.construct()
    qr_entry.fullUrl = 'urn:us-mn-state-health-medss-questionnaireresponse:' + temp.case_id.item()
    qr_entry.resource = qr_rg
    bundle_list.append(qr_entry)

    #Create bundle from list of entries
    patient_bundle.entry = bundle_list
    fhir_message = patient_bundle.json()

    #Print the final bundle as an example
    place = [i for i, x in enumerate(patients) if x == patid][0]

    if place == len(patients) - 1:
        display(json.loads(fhir_message))

    #Code to send below
    send_fhir_message('https://dex.web.health.state.mn.us', fhir_message)